<a href="https://colab.research.google.com/github/SonOfADev/memes_vs_notes/blob/master/dnn_pytorch_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import numpy as np
import PIL
from PIL import Image
import os
 
folder = "/content/drive/My Drive/Memes_vs_Notes/"
classes = ["Memes", "Notes"]

In [ ]:
from sklearn.utils import shuffle
import cv2
def load_data(folder, classes, IMG_HEIGHT, IMG_WIDTH):
    data = []
    labels = []
    label = -1
    for y in classes:
        label += 1
        path = folder + str(y)+"/"
        for x in os.listdir(path):
            img = cv2.imread(path + str(x))
            img = cv2.resize(img,(IMG_WIDTH, IMG_HEIGHT))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            data.append(img)      
            labels.append(label)
    dataset = np.array([np.array(x) for x in data])
    dataset, labels = shuffle (dataset, labels, random_state=0) 
    dataset = torch.Tensor(dataset)
    labels = torch.Tensor(labels)  
     a = np.array([1,2,3,4])
a[-4:]
    return dataset, labels


In [ ]:
IMG_HEIGHT = 50
IMG_WIDTH = 50
data, labels = load_data(folder, classes, IMG_HEIGHT, IMG_WIDTH)

In [ ]:
device =  torch.device("cpu")
data = data/255
data.to(device)
labels.to(device)

data.shape, labels.shape

(torch.Size([1600, 50, 50]), torch.Size([1600]))

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(2500, 3200),
            nn.ReLU()
        )
        self.layer2 = nn.Linear(3200, 1600)
        self.layer4 = nn.Linear (1600, 800)
        self.layer3 = nn.Linear(800,1)

    def forward(self,x): 
        x = x.reshape(-1, 2500)
        out = self.layer1(x)
        #res = out
        out = F.relu(self.layer2(out))
        #out = F.relu(res + self.layer2(out))
        out = F.relu(self.layer4(out))
        out = self.layer3(out)
        return out


In [ ]:
model = DNN()
learning_rate = 0.0001
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:

train_data = data[:1000]
train_labels = labels[:1000]
valid_data = data[1000:1300]
valid_labels = labels[1000:1300]
test_data = data[1300:1600]
test_labels = labels[1300:1600]
np.shape(train_data)

# train_data = train_data.to(device)
# train_labels = train_labels.to(device)

# valid_data = valid_data.to(device)
# valid_lables = valid_labels.to(device)

# test_data = test_data.to(device)
# test_labels = test_labels.to(device)
train_data.shape

torch.Size([1000, 50, 50])

In [ ]:
#train_labels = train_labels.type(torch.LongTensor) 
train_labels = train_labels.unsqueeze(-1)
valid_labels = valid_labels.unsqueeze(-1)
test_labels = test_labels.unsqueeze(-1)
train_labels.type(), train_labels.shape

('torch.FloatTensor', torch.Size([1000, 1]))

In [ ]:
import torch.nn.functional as F
num_epochs = 10
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    total = 0
    correct = 0
    #for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        #data_iter = train_data[:i]
        #labels_iter = labels[:i]
    outputs = model(train_data)
    loss = criterion(outputs, train_labels)
    loss_list.append(loss.item())

    # Backprop and perform Adam optimisation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Track the accuracy
        
    with torch.no_grad():
        #outputs = net(images)
        outputs_valid = model(valid_data)
        valid_loss = criterion(outputs_valid, valid_labels)

        predicted_valid = (F.sigmoid(outputs_valid.data)>0.5)

        predicted = (F.sigmoid(outputs.data)>0.5)

        total = train_labels.size(0)
        correct = (predicted == train_labels).sum().item()

        total_valid = valid_labels.size(0)
        correct_valid = (predicted_valid == valid_labels).sum().item()

    print("training accuracy %d %%" %(100*correct/total ))
    print('validation Accuracy: %d %%' % (100 * correct_valid / total_valid))
    print(f"test loss = {loss}")
    print(f"validation loss = {valid_loss}") 

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


training accuracy 52 %
validation Accuracy: 59 %
test loss = 0.6923733353614807
validation loss = 0.66102534532547
training accuracy 54 %
validation Accuracy: 79 %
test loss = 0.6657779812812805
validation loss = 0.6396945118904114
training accuracy 77 %
validation Accuracy: 84 %
test loss = 0.6404204368591309
validation loss = 0.6133846044540405
training accuracy 85 %
validation Accuracy: 81 %
test loss = 0.6104450821876526
validation loss = 0.5774989724159241
training accuracy 82 %
validation Accuracy: 82 %
test loss = 0.574353814125061
validation loss = 0.5422399044036865
training accuracy 83 %
validation Accuracy: 84 %
test loss = 0.535377562046051
validation loss = 0.5075803399085999
training accuracy 85 %
validation Accuracy: 82 %
test loss = 0.4936813414096832
validation loss = 0.4667215347290039
training accuracy 83 %
validation Accuracy: 84 %
test loss = 0.4526608884334564
validation loss = 0.4437885582447052
training accuracy 85 %
validation Accuracy: 80 %
test loss = 0.41881